In [1]:
import pandas as pd
import collections
from scipy import stats

In [2]:
df = pd.read_csv("../data/Dataframe_full.tsv", sep="\t", index_col=0)
df = df[pd.notnull(df["St1_trans"])]

In [3]:
fitness = collections.OrderedDict([
    ('St1', 2.2),
    ('St2', 7.5),
    ('St3', 4.2),
    ('St4', 6.5),
    ('St5', 0.3)
])

In [4]:
X = df[[strain + "_trans" for strain in fitness]]
X.index = df["NCBI_GENE_ID"]
X = X[pd.notnull(X).all(1)]
X = X[~(X>10000).any(1)]

In [5]:
def regress(row):
    res = stats.linregress(list(row), list(fitness.values()))
    return (res.slope, res.stderr)

coefs = X.apply(regress, axis=1)

In [6]:
result_df = pd.DataFrame(index=X.index)
result_df["mean"] = coefs.map(lambda x: x[0])
result_df["lower"] = coefs.map(lambda x: x[0] - x[1])
result_df["upper"] = coefs.map(lambda x: x[0] + x[1])
result_df["stderr"] = coefs.map(lambda x: x[1])

In [7]:
result_df.to_csv("../data/gene_transciptomics_coefficients.tsv", sep="\t")